In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow import keras

image_size = (480, 640)
batch_size = 32


train_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/train_data"
test_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/test_data"
valid_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/validation_data"


num_classes = 12

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(480, 640),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')


base_model = keras.applications.vgg16.VGG16(
    weights="imagenet",
    input_shape=(480, 640, 3),
    include_top=False)


for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(num_classes, activation='softmax')
    ])




Found 1017 images belonging to 12 classes.
Found 1002 images belonging to 12 classes.
Found 4331 images belonging to 12 classes.


In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 15, 20, 512)       14714688  
                                                                 
 flatten_5 (Flatten)         (None, 153600)            0         
                                                                 
 dense_10 (Dense)            (None, 128)               19660928  
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 12)                1548      
                                                                 
Total params: 34,377,164
Trainable params: 19,662,476
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001, 
    patience=3, 
    verbose=1, 
    mode='auto',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[earlystop_callback]
)

Epoch 1/30
 2/31 [>.............................] - ETA: 1:03:54 - loss: 13.1396 - accuracy: 0.0938